In [23]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import sqlite3 as lite
import os
import sys

In [10]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.124 Safari/537.36'}

In [12]:
cities = {'tx' : "austin"
          , 'ca' : 'los-angeles'
          , 'ga' : 'atlanta'
          , 'ma' : 'boston'
          , 'nc' : 'charlotte'
          , 'tx' : 'dallas'
          , 'co' : 'denver'
          , 'tx' : 'houston'
          , 'fl' : 'miami'
          , 'mn' : 'minneapolis'
          , 'ny' : 'new-york'
          , 'pa' : 'philadelphia'
          , 'az' : 'phoenix'
          , 'or' : 'portland'
          , 'ca' : 'san-francisco'
          , 'ca' : 'san-diego'
          , 'wa' : 'seattle'
          , 'dc' : 'washington'
         }

In [24]:
# CRAWL ALL THE PAGES OF INTERST

datadir = 'crawl/'
if not(os.path.isdir(datadir)):
    os.makedirs(datadir)

for k,v in cities.iteritems():
    running = True
    page = 1
    print ''
    print k,v
    while running:
        url = "https://dogvacay.com/dog-boarding--" + k + "--" + v + "?p="+str(page)
        filename = datadir + k + '-' + v + '-' + str(page) + '.htm'
        if not(os.path.isfile(filename)):
            sys.stdout.write('-')
            r = requests.get(url, headers=headers)
            time.sleep(1)
            f = open(filename, 'w')
            f.write(r.text.encode('ascii', 'replace'))
            f.close()
            data = r.text
        else:
            sys.stdout.write('.')
            f = open(filename, 'r')
            data = f.read()
            f.close()
        soup = BeautifulSoup(data)
        pagination_links = soup.findAll('a', {'class': 'pagination-link'})
        running = False
        for pl in pagination_links:
            if pl.text.find('Next') == 0:
                running = True
        page+=1



mn minneapolis
.......
ma boston
.............
wa seattle
.........
tx houston
...........
ca san-diego
...........
nc charlotte
.......
dc washington
...........................
ny new-york
...........................................--------------------
pa philadelphia
------------
ga atlanta
------------
co denver
-----------
az phoenix
---------
fl miami
---------
or portland
----------

In [36]:
# Data extraction phase

times = []
fees = []
reviews = []
repeats = []
city = []
service = []
rating = []



for src in os.listdir(datadir):
    filename = datadir + src
    print 'Processing: ' + filename
    f = open(filename, 'r')
    data = f.read()
    f.close()
    soup = BeautifulSoup(data)
    sitters = soup.findAll('div', {'class': 'card-content'})
    for sitter in sitters:
        rtwrap = sitter.findAll('div', {'class': 'icon-wrapper'})
        if len(rtwrap) > 0:
            response_time = rtwrap[0].text.strip('\n')[19:]
        else:
            response_time = -1
        fee = sitter.findAll('div', {'class': 'price price--primary u-text-center'})
        review = sitter.find('span', {'class': 'vcard-review'})
        if review is not None and len(review) > 0:
            review = int(review.text.replace('Reviews', '').replace('Review', '').strip())
        else:
            review = 0
        repeat = sitter.findAll('div', {'class': 'icon-wrapper'})
        if len(repeat) == 2:
            repeat = repeat[1].text.strip('\n')[1]
        else:
            repeat = 0
        services = sitter.findAll('div', {'class': 'vcard-line-item'})
        full_star = sitter.findAll('i', {'class': 'ck-star'})
        ratings = len(full_star)
        half_star = sitter.findAll('i', {'class': 'ck-star-half'})
        if len(half_star) > 0:
            ratings += .5
        fees.append(fee[0].text.strip('\n$')[:2])
        city.append(v)
        times.append(response_time)
        reviews.append(review)
        repeats.append(repeat)
        service.append(services[-1].text.strip('\n'))
        rating.append(ratings)

df = pd.DataFrame({'city': city, 'fee': fees, 'response_time': times, 'review' : reviews, 'repeat' : repeats, 'services' : service, 'ratings': rating})

Processing: crawl/az-phoenix-1.htm
Processing: crawl/az-phoenix-2.htm
Processing: crawl/az-phoenix-3.htm
Processing: crawl/az-phoenix-4.htm
Processing: crawl/az-phoenix-5.htm
Processing: crawl/az-phoenix-6.htm
Processing: crawl/az-phoenix-7.htm
Processing: crawl/az-phoenix-8.htm
Processing: crawl/az-phoenix-9.htm
Processing: crawl/ca-san-diego-1.htm
Processing: crawl/ca-san-diego-10.htm
Processing: crawl/ca-san-diego-11.htm
Processing: crawl/ca-san-diego-2.htm
Processing: crawl/ca-san-diego-3.htm
Processing: crawl/ca-san-diego-4.htm
Processing: crawl/ca-san-diego-5.htm
Processing: crawl/ca-san-diego-6.htm
Processing: crawl/ca-san-diego-7.htm
Processing: crawl/ca-san-diego-8.htm
Processing: crawl/ca-san-diego-9.htm
Processing: crawl/co-denver-1.htm
Processing: crawl/co-denver-10.htm
Processing: crawl/co-denver-11.htm
Processing: crawl/co-denver-2.htm
Processing: crawl/co-denver-3.htm
Processing: crawl/co-denver-4.htm
Processing: crawl/co-denver-5.htm
Processing: crawl/co-denver-6.htm
Pr

In [37]:
df.shape

(3089, 7)

In [38]:
df.head()

,city,fee,ratings,repeat,response_time,review,services
0,minneapolis,20,5,6,Few Days,20,"Boarding, Sitting, Daycare, Checkups and Visit..."
1,minneapolis,31,0,0,Few Hours,0,"Boarding, Bathing, and Pick up and Drop off"
2,minneapolis,25,5,4,Day,9,Boarding and Pick up and Drop off
3,minneapolis,25,5,4,Few Hours,4,Boarding and Daycare
4,minneapolis,25,0,0,Few Hours,0,Boarding


In [39]:
# SAVE TO CSV FILE

df.to_csv('dog-vacay.csv', sep='\t')